In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [4]:
haar_data = cv2.CascadeClassifier('C:/Users/A715-71G/Desktop/Haar/haarcascade_frontalface_default.xml')


In [5]:
capture = cv2.VideoCapture(0) #initialize camera
without_mask = []
while True:
    flag, img = capture.read() #read video frame by frame and return true/false and one frame at a time
    if flag: #check camera available or not
        faces = haar_data.detectMultiScale(img)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 255), 4) #drawing rectangle
            face = img[y:y+h, x:x+w, :] #slicing only face from the frame
            face = cv2.resize(face, (50, 50)) #resizing all faces to size 50x50, so that all images are at the same size
            print(len(without_mask))
            if len(without_mask) < 200: #only store 200 images:
                without_mask.append(face)
        cv2.imshow('result', img) #show image
        if cv2.waitKey(2) == 27 or len(without_mask) == 200: #break loop if press Escape or 200 images are stored
                break
capture.release()  #release camera object
cv2.destroyAllWindows()#close all windows
without_mask = np.array(without_mask).reshape(200, 50 * 50 * 3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [12]:
capture = cv2.VideoCapture(0) #initialize camera
with_mask = []
while True:
    flag, img = capture.read() #read video frame by frame and return true/false and one frame at a time
    if flag: #check camera available or not
        faces = haar_data.detectMultiScale(img)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 255), 4) #drawing rectangle
            face = img[y:y+h, x:x+w, :] #slicing only face from the frame
            face = cv2.resize(face, (50, 50)) #resizing all faces to size 50x50, so that all images are at the same size
            print(len(with_mask))
            if len(with_mask) < 200: #only store 200 images:
                with_mask.append(face)
        cv2.imshow('result', img) #show image
        if cv2.waitKey(2) == 27 or len(with_mask) == 200: #break loop if press Escape or 200 images are stored
                break
capture.release()  #release camera object
cv2.destroyAllWindows()#close all windows
with_mask = np.array(with_mask).reshape(200, 50 * 50 * 3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [13]:
np.save('with_mask.npy', with_mask)#save images into npy file
np.save('without_mask.npy', without_mask)#save images into npy file

In [14]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [15]:
with_mask.shape


(200, 7500)

In [16]:
without_mask.shape

(200, 7500)

In [17]:
with_mask = with_mask.reshape(200, 50* 50 * 3)
without_mask = without_mask.reshape(200, 50 * 50 * 3)

In [18]:
X = np.r_[with_mask, without_mask] #concatenate data into single array

In [19]:
X.shape

(400, 7500)

In [20]:
labels = np.zeros(X.shape[0]) #create and assign first 200 indexes as zero
labels[200:] = 1.0            #and the 200 next indexes as one
#label --> 0 : with mask, 1: without_mask
names = {0 : 'Mask', 1: 'No Mask'}


In [21]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [22]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size = 0.25)

In [23]:
x_train.shape

(300, 7500)

In [24]:
#x_test = pca.transform(x_test)
svm = SVC()
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)

In [25]:
accuracy_score(y_test, y_pred)

0.99

In [26]:
haar_data = cv2.CascadeClassifier('C:/Users/A715-71G/Desktop/Haar/haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0)
while True:
    flag, img = capture.read() #read video frame by frame and return true/false and one frame at a time
    if flag: #check camera available or not
        faces = haar_data.detectMultiScale(img)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 255), 4) #drawing rectangle
            face = img[y:y+h, x:x+w, :] #slicing only face from the frame
            face = cv2.resize(face, (50, 50)) #resizing all faces to size 50x50, so that all images are at the same size
            face = face.reshape(1, -1)
            pred = svm.predict(face)
            n = names[int(pred)]
            print(n)
        cv2.imshow('result', img) #show image
        if cv2.waitKey(2) == 27:  #break loop if press Escape or 200 images are stored
                break
capture.release()
cv2.destroyAllWindows()

Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
Mask
No Mask
No Mask
Mask
Mask
